In [1]:
#Import libraries
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, roc_auc_score, classification_report

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

import keras
import tensorflow as tf

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os

In [2]:
path = 'Dataset2/'
im_size = 350

In [3]:
street_types = os.listdir(path)
print(street_types)
print("Types of streets found: ", len(street_types))

['clean', 'litter', 'recycle']
Types of streets found:  3


In [4]:
streets = []
for item in street_types:
    all_streets = os.listdir(path + '/' + item) #get all the file names
    #print(all_streets)
    
    #Add them to the list
    for street in all_streets:
        streets.append((item, str(path + '/' + item) + '/' + street))
        #print(streets[:1])

In [5]:
#Build a dataframe
streets_df = pd.DataFrame(data=streets, columns=['street type', 'image'])
#print(streets_df.head())
#print(streets_df.tail())

In [6]:
#Let´s check how many samples for each category are present
print("Total number of streets in the Dataset: ", len(streets_df))

Total number of streets in the Dataset:  5405


In [7]:
street_count = streets_df['street type'].value_counts()
print("streets in each category: ")
print(street_count)

streets in each category: 
street type
recycle    1875
clean      1825
litter     1705
Name: count, dtype: int64


In [8]:
images = []
labels = []

for label in street_types:
    data_path = os.path.join(path, label)
    filenames = os.listdir(data_path)

    for filename in filenames:
        img_path = os.path.join(data_path, filename)
        img = load_img(img_path, target_size=(im_size, im_size))  # Carrega a imagem
        img_array = img_to_array(img)  # Converte a imagem para um array numpy
        images.append(img_array)
        labels.append(label)

In [10]:
#Transform the image to a numpy type

images = np.array(images)
images = images.astype('float32')/255.0
images.shape

(5405, 350, 350, 3)

In [116]:
y = streets_df['street type'].values
#print(y[:5])

y_labelenconder = LabelEncoder ()
y = y_labelenconder.fit_transform (y)
#print(y)

In [117]:
images, y = shuffle(images, y, random_state=1)
train_x, test_x, train_y, test_y = train_test_split(images, y, test_size=0.05, random_state=415)

print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(5134, 350, 350, 3)
(271, 350, 350, 3)
(5134,)
(271,)


In [118]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(im_size, im_size, 3)),
    keras.layers.Dense(256, activation=tf.nn.tanh),
    keras.layers.Dense(128, activation=tf.nn.relu),  # Camada densa adicional
    keras.layers.Dense(64, activation=tf.nn.relu),   # Outra camada densa adicional
    keras.layers.Dense(3, activation=tf.nn.softmax)
])

In [119]:
model.compile(optimizer = Adam(learning_rate = 0.01),
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

In [120]:
model.fit(train_x, 
          train_y, 
          epochs=10, 
          batch_size=64, 
          validation_data=(test_x, test_y))

Epoch 1/10
81/81 [==============================] - 47s 521ms/step - loss: 1.2100 - accuracy: 0.3342 - val_loss: 1.1000 - val_accuracy: 0.3653
Epoch 2/10
81/81 [==============================] - 32s 391ms/step - loss: 1.1019 - accuracy: 0.3362 - val_loss: 1.0957 - val_accuracy: 0.3616
Epoch 3/10
81/81 [==============================] - 32s 391ms/step - loss: 1.0966 - accuracy: 0.3588 - val_loss: 1.0918 - val_accuracy: 0.3653
Epoch 4/10
81/81 [==============================] - 32s 391ms/step - loss: 1.0986 - accuracy: 0.3432 - val_loss: 1.0927 - val_accuracy: 0.3653
Epoch 5/10
81/81 [==============================] - 31s 385ms/step - loss: 1.0984 - accuracy: 0.3422 - val_loss: 1.0933 - val_accuracy: 0.3653
Epoch 6/10
81/81 [==============================] - 30s 366ms/step - loss: 1.0985 - accuracy: 0.3416 - val_loss: 1.0970 - val_accuracy: 0.3653
Epoch 7/10
81/81 [==============================] - 30s 368ms/step - loss: 1.0988 - accuracy: 0.3391 - val_loss: 1.0968 - val_accuracy: 0.3653

In [121]:
y_probs = model.predict(test_x)
y_pred = np.argmax(y_probs, axis=1)

9/9 [==============================] - 1s 95ms/step


In [122]:
confusion = confusion_matrix(test_y, y_pred)
print("Matriz de Confusão:")
print(confusion)

report = classification_report(test_y, y_pred, target_names=street_types, zero_division=1)
print(report)

# Calcule o AUC para cada classe
for i in range(len(street_types)):
    auc = roc_auc_score(test_y == i, y_probs[:, i])
    print(f"AUC para classe {street_types}: {auc}")

Matriz de Confusão:
[[ 0  0 98]
 [ 0  0 74]
 [ 0  0 99]]
              precision    recall  f1-score   support

       clean       1.00      0.00      0.00        98
      litter       1.00      0.00      0.00        74
     recycle       0.37      1.00      0.54        99

    accuracy                           0.37       271
   macro avg       0.79      0.33      0.18       271
weighted avg       0.77      0.37      0.20       271

AUC para classe ['clean', 'litter', 'recycle']: 0.5
AUC para classe ['clean', 'litter', 'recycle']: 0.5
AUC para classe ['clean', 'litter', 'recycle']: 0.5
